# Handling Missing Data With Pandas

Pandas borrows all the capabilities from numpy selection + adds a number of convenient methods to handle missing values. Let's see one at a time

In [1]:
import numpy as np
import pandas as pd

## Pandas Utility functions
Similary to **numpy** , pandas also has a few utility functions to identify and detect null values:
* isnull()
* isna()
* notnull()
* notna()

In [9]:
pd.isnull(np.nan),
pd.isnull(None),
pd.isna(np.nan)

True

The opposite ones also exist:

In [26]:
pd.notnull(np.nan),
pd.notna(None),
pd.notnull(3)

True

These functions also work with **Series** and **DataFrame**

In [20]:
df = pd.Series([1, np.nan, 3])

In [21]:
pd.isnull(df) # or pd.isnull([1,np.nan, 3])

0    False
1     True
2    False
dtype: bool

In [22]:
pd.notnull(df)

0     True
1    False
2     True
dtype: bool

In [25]:
df= pd.DataFrame({'Column A': [1, np.nan, 3],
                 'Column B': [np.nan, 2, 3],
                 'Column C' : [2, 3, np.nan]})
df

,Column A,Column B,Column C
0,1.0,NaN,2.0
1,NaN,2.0,3.0
2,3.0,3.0,NaN


In [24]:
pd.isnull(df)

,Column A,Column B,Column C
0,False,True,False
1,True,False,False
2,False,False,True


### Pandas Operations with missing values

Pandas manages missing values more gracefully than numpy, **nan** will no longer behave as **viruses** and operations will just ignore them completely.

In [29]:
df

,Column A,Column B,Column C
0,1.0,NaN,2.0
1,NaN,2.0,3.0
2,3.0,3.0,NaN


In [41]:
df.isnull(), df.notnull(), df.isnull().sum(), df.isna().sum()

(   Column A  Column B  Column C
 0     False      True     False
 1      True     False     False
 2     False     False      True,
    Column A  Column B  Column C
 0      True     False      True
 1     False      True      True
 2      True      True     False,
 Column A    1
 Column B    1
 Column C    1
 dtype: int64,
 Column A    1
 Column B    1
 Column C    1
 dtype: int64)

In [43]:
df[df.isnull()], df[df.notnull()]

(   Column A  Column B  Column C
 0       NaN       NaN       NaN
 1       NaN       NaN       NaN
 2       NaN       NaN       NaN,
    Column A  Column B  Column C
 0       1.0       NaN       2.0
 1       NaN       2.0       3.0
 2       3.0       3.0       NaN)

### Dropping null values
* dropnull()
* dropna()

Boolean selection  + notnull() seems a little bit verbose and repetitive. And as we said before: any repetitive task will probaly have a better, more DRY way. In this case, we can use the dropna method.

In [47]:
df

,Column A,Column B,Column C
0,1.0,NaN,2.0
1,NaN,2.0,3.0
2,3.0,3.0,NaN


In [46]:
df.dropna()

,Column A,Column B,Column C


As you can see, the dropna() deleted all rows if existing na/ null values. So with DataFrame, there are more things to consider, because you can't drop the single value

In [56]:
df = pd.DataFrame({'Column A':[1, np.nan, 30, np.nan],
                   'Column B':[2, 8, 31, np.nan],
                   'Column C':[np.nan, 9, 32, 100],
                   'Column D':[5, 8 ,34, 110]})

In [64]:
df.shape, df.size, df.info() , 
df.isnull(), df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 256.0 bytes


(   Column A  Column B  Column C  Column D
 0     False     False      True     False
 1      True     False     False     False
 2     False     False     False     False
 3      True      True     False     False,
 Column A    2
 Column B    1
 Column C    1
 Column D    0
 dtype: int64)

If you use **dropna()** method, all rows with null value will be deleted, and you can also drop columns with null value by using **axis** attribute

In [66]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [65]:
df.dropna(axis=1)

,Column D
0,5
1,8
2,34
3,110


In case you want to control this behavior, you can use the **how** parameter with two options
* all : if all value is null, drop that row or column
* any (default option): if any value is null, drop that row and column

In [67]:
df.dropna(how='all')

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [68]:
df.dropna(how='any')

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


You can also use the **thresh** parameter to indicate a threshold (a minimum number) of non-null values for the row/column to be kept. **Axis** parameter indicates the axis will be kept. **row is default**

In [69]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [70]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [71]:
df.dropna(thresh=3, axis='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


### Filling null values

Sometimes instead than dropping the null values, we might need to replace them with some other values. This highly depends on your context and the dataset you're currently working. There're different methods and mechanisms and you can then apply them to your own problem:
#### fillna()

In [72]:
df.fillna(0), df.fillna(df.mean())

(   Column A  Column B  Column C  Column D
 0       1.0       2.0       0.0         5
 1       0.0       8.0       9.0         8
 2      30.0      31.0      32.0        34
 3       0.0       0.0     100.0       110,
    Column A   Column B  Column C  Column D
 0       1.0   2.000000      47.0         5
 1      15.5   8.000000       9.0         8
 2      30.0  31.000000      32.0        34
 3      15.5  13.666667     100.0       110)

#### Filling nulls with contiguous (close) values
* ffill : front value. If it's the first value, the value fill will still be NA 
* bfill : following value If it's the last value, the value fill will still be NA

In [79]:
df,df.fillna(method='ffill'),df.fillna(method='bfill')

(   Column A  Column B  Column C  Column D
 0       1.0       2.0       NaN         5
 1       NaN       8.0       9.0         8
 2      30.0      31.0      32.0        34
 3       NaN       NaN     100.0       110,
    Column A  Column B  Column C  Column D
 0       1.0       2.0       NaN         5
 1       1.0       8.0       9.0         8
 2      30.0      31.0      32.0        34
 3      30.0      31.0     100.0       110,
    Column A  Column B  Column C  Column D
 0       1.0       2.0       9.0         5
 1      30.0       8.0       9.0         8
 2      30.0      31.0      32.0        34
 3       NaN       NaN     100.0       110)

In [83]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In DataFrame, you can use the **axis** to fill the values.
* axis = 0, by row
* axis = 1, by column

In [86]:
df, df.fillna(method='ffill', axis=0),df.fillna(method='ffill', axis=1)

(   Column A  Column B  Column C  Column D
 0       1.0       2.0       NaN         5
 1       NaN       8.0       9.0         8
 2      30.0      31.0      32.0        34
 3       NaN       NaN     100.0       110,
    Column A  Column B  Column C  Column D
 0       1.0       2.0       NaN         5
 1       1.0       8.0       9.0         8
 2      30.0      31.0      32.0        34
 3      30.0      31.0     100.0       110,
    Column A  Column B  Column C  Column D
 0       1.0       2.0       2.0       5.0
 1       NaN       8.0       9.0       8.0
 2      30.0      31.0      32.0      34.0
 3       NaN       NaN     100.0     110.0)

### Checking if there are NAs